In [1]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
import numpy as np
import gym

# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?
render = True

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid

model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)

print(model['W1'].shape)
print(model['W2'].shape)

grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
    """ backward pass. (eph is array of intermediate hidden states) """
    dW2 = np.dot(eph.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[eph <= 0] = 0 # backpro prelu
    dW1 = np.dot(dh.T, epx)
    return {'W1':dW1, 'W2':dW2}

(200, 6400)
(200,)


In [ ]:

env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
while True:
    if render: env.render()

    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
        episode_number += 1

        # stack together all inputs, hidden states, action gradients, and rewards for this episode
        epx = np.vstack(xs)
        eph = np.vstack(hs)
        epdlogp = np.vstack(dlogps)
        epr = np.vstack(drs)
        xs,hs,dlogps,drs = [],[],[],[] # reset array memory

        # compute the discounted reward backwards through time
        discounted_epr = discount_rewards(epr)
        # standardize the rewards to be unit normal (helps control the gradient estimator variance)
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)

        epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
        grad = policy_backward(eph, epdlogp)
        for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

        # perform rmsprop parameter update every batch_size episodes
        if episode_number % batch_size == 0:
            for k,v in model.items():
                g = grad_buffer[k] # gradient
                rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
                model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
                grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

        # boring book-keeping
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
        #if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_x = None


resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000
resetting env. episode reward total was -21.000000. running mean: -21.000000

resetting env. episode reward total was -17.000000. running mean: -20.638863
resetting env. episode reward total was -20.000000. running mean: -20.632475
resetting env. episode reward total was -21.000000. running mean: -20.636150
resetting env. episode reward total was -20.000000. running mean: -20.629789
resetting env. episode reward total was -21.000000. running mean: -20.633491
resetting env. episode reward total was -20.000000. running mean: -20.627156
resetting env. episode reward total was -21.000000. running mean: -20.630884
resetting env. episode reward total was -21.000000. running mean: -20.634575
resetting env. episode reward total was -20.000000. running mean: -20.628230
resetting env. episode reward total was -21.000000. running mean: -20.631947
resetting env. episode reward total was -20.000000. running mean: -20.625628
resetting env. episode reward total was -20.000000. running mean: -20.619372
resetting env. episode reward total was -21.000000. running mean: -20.623178

resetting env. episode reward total was -21.000000. running mean: -20.552122
resetting env. episode reward total was -21.000000. running mean: -20.556601
resetting env. episode reward total was -21.000000. running mean: -20.561035
resetting env. episode reward total was -20.000000. running mean: -20.555425
resetting env. episode reward total was -20.000000. running mean: -20.549870
resetting env. episode reward total was -20.000000. running mean: -20.544372
resetting env. episode reward total was -21.000000. running mean: -20.548928
resetting env. episode reward total was -19.000000. running mean: -20.533439
resetting env. episode reward total was -21.000000. running mean: -20.538104
resetting env. episode reward total was -18.000000. running mean: -20.512723
resetting env. episode reward total was -21.000000. running mean: -20.517596
resetting env. episode reward total was -20.000000. running mean: -20.512420
resetting env. episode reward total was -21.000000. running mean: -20.517296

resetting env. episode reward total was -21.000000. running mean: -20.496387
resetting env. episode reward total was -21.000000. running mean: -20.501423
resetting env. episode reward total was -18.000000. running mean: -20.476409
resetting env. episode reward total was -21.000000. running mean: -20.481645
resetting env. episode reward total was -21.000000. running mean: -20.486828
resetting env. episode reward total was -20.000000. running mean: -20.481960
resetting env. episode reward total was -19.000000. running mean: -20.467140
resetting env. episode reward total was -20.000000. running mean: -20.462469
resetting env. episode reward total was -20.000000. running mean: -20.457844
resetting env. episode reward total was -21.000000. running mean: -20.463266
resetting env. episode reward total was -18.000000. running mean: -20.438633
resetting env. episode reward total was -21.000000. running mean: -20.444247
resetting env. episode reward total was -20.000000. running mean: -20.439804

resetting env. episode reward total was -21.000000. running mean: -20.391851
resetting env. episode reward total was -21.000000. running mean: -20.397933
resetting env. episode reward total was -21.000000. running mean: -20.403953
resetting env. episode reward total was -20.000000. running mean: -20.399914
resetting env. episode reward total was -21.000000. running mean: -20.405915
resetting env. episode reward total was -21.000000. running mean: -20.411856
resetting env. episode reward total was -21.000000. running mean: -20.417737
resetting env. episode reward total was -21.000000. running mean: -20.423560
resetting env. episode reward total was -19.000000. running mean: -20.409324
resetting env. episode reward total was -20.000000. running mean: -20.405231
resetting env. episode reward total was -21.000000. running mean: -20.411179
resetting env. episode reward total was -21.000000. running mean: -20.417067
resetting env. episode reward total was -21.000000. running mean: -20.422896

...

resetting env. episode reward total was -21.000000. running mean: -19.759009
resetting env. episode reward total was -20.000000. running mean: -19.761419
resetting env. episode reward total was -21.000000. running mean: -19.773805
resetting env. episode reward total was -20.000000. running mean: -19.776067
resetting env. episode reward total was -21.000000. running mean: -19.788306
resetting env. episode reward total was -19.000000. running mean: -19.780423
resetting env. episode reward total was -20.000000. running mean: -19.782619
resetting env. episode reward total was -19.000000. running mean: -19.774793
resetting env. episode reward total was -20.000000. running mean: -19.777045
resetting env. episode reward total was -19.000000. running mean: -19.769275
resetting env. episode reward total was -19.000000. running mean: -19.761582
resetting env. episode reward total was -21.000000. running mean: -19.773966
resetting env. episode reward total was -21.000000. running mean: -19.786226

resetting env. episode reward total was -20.000000. running mean: -19.745644
resetting env. episode reward total was -19.000000. running mean: -19.738187
resetting env. episode reward total was -19.000000. running mean: -19.730805
resetting env. episode reward total was -20.000000. running mean: -19.733497
resetting env. episode reward total was -19.000000. running mean: -19.726162
resetting env. episode reward total was -20.000000. running mean: -19.728901
resetting env. episode reward total was -19.000000. running mean: -19.721612
resetting env. episode reward total was -20.000000. running mean: -19.724396
resetting env. episode reward total was -20.000000. running mean: -19.727152
resetting env. episode reward total was -18.000000. running mean: -19.709880
resetting env. episode reward total was -20.000000. running mean: -19.712781
resetting env. episode reward total was -18.000000. running mean: -19.695653
resetting env. episode reward total was -17.000000. running mean: -19.668697

resetting env. episode reward total was -20.000000. running mean: -19.692044
resetting env. episode reward total was -21.000000. running mean: -19.705123
resetting env. episode reward total was -21.000000. running mean: -19.718072
resetting env. episode reward total was -20.000000. running mean: -19.720891
resetting env. episode reward total was -20.000000. running mean: -19.723683
resetting env. episode reward total was -18.000000. running mean: -19.706446
resetting env. episode reward total was -18.000000. running mean: -19.689381
resetting env. episode reward total was -17.000000. running mean: -19.662487
resetting env. episode reward total was -20.000000. running mean: -19.665863
resetting env. episode reward total was -21.000000. running mean: -19.679204
resetting env. episode reward total was -19.000000. running mean: -19.672412
resetting env. episode reward total was -18.000000. running mean: -19.655688
resetting env. episode reward total was -21.000000. running mean: -19.669131

resetting env. episode reward total was -20.000000. running mean: -19.667926
resetting env. episode reward total was -21.000000. running mean: -19.681247
resetting env. episode reward total was -21.000000. running mean: -19.694434
resetting env. episode reward total was -20.000000. running mean: -19.697490
resetting env. episode reward total was -21.000000. running mean: -19.710515
resetting env. episode reward total was -21.000000. running mean: -19.723410
resetting env. episode reward total was -21.000000. running mean: -19.736176
resetting env. episode reward total was -19.000000. running mean: -19.728814
resetting env. episode reward total was -19.000000. running mean: -19.721526
resetting env. episode reward total was -21.000000. running mean: -19.734311
resetting env. episode reward total was -18.000000. running mean: -19.716967
resetting env. episode reward total was -19.000000. running mean: -19.709798
resetting env. episode reward total was -19.000000. running mean: -19.702700

resetting env. episode reward total was -21.000000. running mean: -19.712016
resetting env. episode reward total was -20.000000. running mean: -19.714895
resetting env. episode reward total was -20.000000. running mean: -19.717747
resetting env. episode reward total was -21.000000. running mean: -19.730569
resetting env. episode reward total was -20.000000. running mean: -19.733263
resetting env. episode reward total was -20.000000. running mean: -19.735931
resetting env. episode reward total was -21.000000. running mean: -19.748571
resetting env. episode reward total was -19.000000. running mean: -19.741086
resetting env. episode reward total was -21.000000. running mean: -19.753675
resetting env. episode reward total was -19.000000. running mean: -19.746138
resetting env. episode reward total was -19.000000. running mean: -19.738677
resetting env. episode reward total was -21.000000. running mean: -19.751290
resetting env. episode reward total was -20.000000. running mean: -19.753777

In [28]:
env.close()